In [305]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing modules**

In [306]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import eli5
import lightgbm as lgb

pd.set_option('display.max_columns', None)
from collections import Counter
from wordcloud import WordCloud
%matplotlib inline
plt.style.use('ggplot')
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
import xgboost
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import absolute
from lightgbm import LGBMRegressor


import warnings
warnings.filterwarnings('ignore')

**Read train and test data**

In [307]:
train = pd.read_csv("/kaggle/input/tmdb-box-office-prediction/train.csv")
test = pd.read_csv("/kaggle/input/tmdb-box-office-prediction/test.csv")

In [308]:
print("Shape of train data : ", train.shape)
print("Shape of test data : ", test.shape)

In [309]:
train.head(2)

We have given data in string format for accessing value by key it's important to convert string into dictionary

In [310]:
# Convert string columns to dictionary
json_columns = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 'Keywords', 'cast', 'crew']

def text_to_dict(df, json_columns = json_columns):
    for col in json_columns:
        df[col] = df[col].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x))
    return df

train = text_to_dict(train)
test = text_to_dict(test)

**belongs_to_collection**

In [311]:
train['belongs_to_collection'][:3]

In [312]:
train['belongs_to_collection'].apply(lambda x: len(x) if x!={} else 0).value_counts()

In [313]:
train['collection'] = train['belongs_to_collection'].apply(lambda x: x[0]['name'] if x!={} else 0)
train['has_collection'] = train['collection'].apply(lambda x: 1 if x!=0 else 0)

test['collection'] = test['belongs_to_collection'].apply(lambda x: x[0]['name'] if x!={} else 0)
test['has_collection'] = test['collection'].apply(lambda x: 1 if x!=0 else 0)

train.drop(['belongs_to_collection'], axis=1, inplace=True)
test.drop(['belongs_to_collection'], axis=1, inplace=True)

**Genres**

In [314]:
train['genres'][:3]

In [315]:
train['genres'].apply(lambda x: len(x) if x!={} else 0).value_counts()

most of movie have 1 to 5 genre but there is also exist some movie that have more then 5 genre so may be thay can be outlier.

In [316]:
train['num_genres'] = train['genres'].apply(lambda x: len(x) if x!={} else 0)
list_of_genres = list(train['genres'].apply(lambda x: [j['name'] for j in x] if x!={} else []).values)
train['all_genres'] = train['genres'].apply(lambda x: ' '.join([j['name'] for j in x]) if x!={} else "" )

top_genres = [m[0] for m in Counter([i for j in list_of_genres for i in j]).most_common(15)]
for g in top_genres:
    train['genres_'+g] = train['all_genres'].apply(lambda x: 1 if g in x else 0)


test['num_genres'] = test['genres'].apply(lambda x: len(x) if x!={} else 0)
test['all_genres'] = test['genres'].apply(lambda x: ' '.join([j['name'] for j in x]) if x!={} else "" )

for g in top_genres:
    test['genres_'+g] = test['all_genres'].apply(lambda x: 1 if g in x else 0)


Here we made special category for most common 15 genre and assign 1 if movie belongs to that genre else 0

In [317]:
plt.figure(figsize=(12, 8))
text = ' '.join(i for j in list_of_genres for i in j)
img = WordCloud(max_font_size=None, background_color='white', collocations=False,
                      width=1200, height=1000).generate(text)
plt.title('Top genres wordcloud')
plt.axis("off")
plt.imshow(img)
plt.show()

Based on wordcloud we can say that most of the movie belongs to Drama, comedy and thriller genre.

**production_companies**

Number of production companies involved for making one movie

In [318]:
train['production_companies'].apply(lambda x: len(x) if x!={} else 0).value_counts()

In [319]:
train['production_companies'].head(2)

In [320]:
train['num_production_companies'] = train['production_companies'].apply(lambda x: len(x) if x!={} else 0)
list_of_pc = train['production_companies'].apply(lambda x: [i['name'] for i in x] if x!={} else []).values
train['all_production_companies'] = train['production_companies'].apply(lambda x : ' '.join( i['name'] for i in x ) if x!={} else "" )

top_pc = [m[0] for m in Counter(i for j in list_of_pc for i in j).most_common(15)]
for g in top_pc:
    train['production_companies_' + g] = train['all_production_companies'].apply(lambda x:1 if g in x else 0)



test['num_production_companies'] = test['production_companies'].apply(lambda x: len(x) if x!={} else 0)
test['all_production_companies'] = test['production_companies'].apply(lambda x : ' '.join( i['name'] for i in x ) if x!={} else "" )

for g in top_pc:
    test['production_companies_' + g] = test['all_production_companies'].apply(lambda x:1 if g in x else 0)

In [321]:
plt.figure(figsize=(12, 8))
text = ' '.join(i for j in list_of_pc for i in j)
img = WordCloud(max_font_size=None, background_color='white', collocations=False,
                      width=1200, height=1000).generate(text)
plt.axis('off')
plt.imshow(img)
plt.show()

**Production Countries**

In [322]:
train['production_countries'][:2]

In [323]:
train['production_countries'].apply(lambda x: len(x) if x!={} else 0).value_counts()

In data there exist some movie that does not belongs to any country so we can say those are outliers.

In [324]:
train['num_production_countries'] = train['production_countries'].apply(lambda x: len(x) if x!={} else 0)
list_of_prod_countries = train['production_countries'].apply(lambda x: [i['name'] for i in x]  if x!={} else []).values
train['all_production_countries'] = train['production_countries'].apply(lambda x: ' '.join(i['name'] for i in x) if x!={} else "")
top_production_countries = [m[0] for m in Counter([i for j in list_of_prod_countries for i in j]).most_common(15)]

for g in top_production_countries:
    train['production_countries_' + g] = train['all_production_countries'].apply(lambda x: 1 if g in x else 0)
    

test['num_production_countries'] = test['production_countries'].apply(lambda x: len(x) if x!={} else 0)
test['all_production_countries'] = test['production_countries'].apply(lambda x: ' '.join(i['name'] for i in x) if x!={} else "")

for g in top_production_countries:
    test['production_countries_' + g] = test['all_production_countries'].apply(lambda x: 1 if g in x else 0)

In [325]:
for i, j in Counter([i for j in list_of_prod_countries for i in j]).most_common(10):
    print(i,(30-len(i))*" ", j/len(list_of_prod_countries)*100)

US involved as production country in more than 76% movie

**Spoken language**

In [326]:
train['spoken_languages'].apply(lambda x: len(x) if x!={} else 0).value_counts()

In [327]:
train['num_spoken_languages'] = train['spoken_languages'].apply(lambda x: len(x) if x!={} else 0)
list_of_spoken_languages = train['spoken_languages'].apply(lambda x: [i['name'] for i in x] if x!={} else "").values
train['all_spoken_languages'] = train['spoken_languages'].apply(lambda x: [' '.join(i['name'] for i in x)] if x!={} else [] )
top_spoken_languages = [m[0] for m in Counter(i for j in list_of_spoken_languages for i in j).most_common(15)]

for g in top_spoken_languages:
    train['spoken_language_'+g] = train['all_spoken_languages'].apply(lambda x: 1 if g in x else 0)
    
test['num_spoken_languages'] = test['spoken_languages'].apply(lambda x: len(x) if x!={} else 0)
test['all_spoken_languages'] = test['spoken_languages'].apply(lambda x: [' '.join(i['name'] for i in x)] if x!={} else [] )

for g in top_spoken_languages:
    test['spoken_language_'+g] = test['all_spoken_languages'].apply(lambda x: 1 if g in x else 0)

In [328]:
for i,j in Counter([i for j in list_of_spoken_languages for i in j]).most_common(10):
    print(i, (70-len(i))*" ", round(j*100/len(list_of_spoken_languages), 2))

in 87% of the movie spoken language is english

**Keywords**

In [329]:
train['Keywords'][:2]

In [330]:
train['Keywords'].apply(lambda x: len(x) if x!={} else 0).value_counts().head(10)

In [331]:
train['num_Keywords'] = train['Keywords'].apply(lambda x: len(x) if x!={} else 0)
list_of_Keywords = train['Keywords'].apply(lambda x: [i['name'] for i in x] if x!={} else []).values
train['all_Keywords'] = train['Keywords'].apply(lambda x: ' '.join(i['name'] for i in x) if x!={} else "")
top_Keywords = [m[0] for m in Counter(i for j in list_of_Keywords for i in j).most_common(30)]

for g in top_Keywords:
    train['Keywords_'+g] = train['all_Keywords'].apply(lambda x: 1 if g in x else 0)
    
    
test['num_Keywords'] = test['Keywords'].apply(lambda x: len(x) if x!={} else 0)
test['all_Keywords'] = test['Keywords'].apply(lambda x: ' '.join(i['name'] for i in x) if x!={} else "")

for g in top_Keywords:
    test['Keywords_'+g] = test['all_Keywords'].apply(lambda x: 1 if g in x else 0)

In [332]:
for i,j in Counter([i for j in list_of_Keywords for i in j]).most_common(20):
    print(i, (50 - len(i))*" ", j)

Most occuring keywords are woman director, independent film, duringcreditsstinger, murder and based on novel

**Cast**

In [333]:
train['cast'][:2]

In [334]:
train['cast'].apply(lambda x:len(x) if x!={} else 0).value_counts()

In [335]:
train['num_cast'] = train['cast'].apply(lambda x: len(x) if x!={} else 0)
list_of_cast = train['cast'].apply(lambda x: [i['name'] for i in x] if x!={} else []).values
train['all_cast'] = train['cast'].apply(lambda x: ' '.join(i['name'] for i in x) if x!={} else "")

top_cast = [m[0] for m in Counter([i for j in list_of_cast for i in j]).most_common(20)]

for g in top_cast:
    train['cast_name_'+g] = train['all_cast'].apply(lambda x: 1 if g in x else 0)
    

test['num_cast'] = test['cast'].apply(lambda x: len(x) if x!={} else 0)
test['all_cast'] = test['cast'].apply(lambda x: ' '.join(i['name'] for i in x) if x!={} else "")

for g in top_cast:
    test['cast_name_'+g] = test['all_cast'].apply(lambda x: 1 if g in x else 0)

Most popular popular cast and number of movies they are in

In [336]:
for i, j in Counter([i for j in list_of_cast for i in j]).most_common(15):
    print(i, (50-len(i))*" ", j)

In [337]:
list_of_gender = train['cast'].apply(lambda x: [i['gender'] for i in x] if x!={} else [])
Counter([i for j in list_of_gender for i in j]).most_common()

In [338]:
train['gender_0_cast'] = train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
train['gender_1_cast'] = train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
train['gender_2_cast'] = train['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))

test['gender_0_cast'] = test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 0]))
test['gender_1_cast'] = test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 1]))
test['gender_2_cast'] = test['cast'].apply(lambda x: sum([1 for i in x if i['gender'] == 2]))

Here we just assigned number of gender in total cast for particular one movie like number of male for x movie from all x cast

In [339]:
list_of_characters = train['cast'].apply(lambda x: [i['character'] for i in x] if x!={} else [])
Counter(i for j in list_of_characters for i in j).most_common(15)

For cast character there are lot of people who play himself/ herself in movie

In [340]:
train['num_cast_characters'] = train['cast'].apply(lambda x: len(x) if x!={} else 0)
train['all_cast_character'] = train['cast'].apply(lambda x: " ".join(i['character'] for i in x) if x!={} else "")
top_cast_characters = [m[0] for m in Counter(i for j in list_of_characters for i in j).most_common(20)]
for g in top_cast_characters:
    train['cast_character_'+g] = train['all_cast_character'].apply(lambda x: 1 if g in x else 0)

test['num_cast_characters'] = test['cast'].apply(lambda x: len(x) if x!={} else 0)
test['all_cast_character'] = test['cast'].apply(lambda x: " ".join(i['character'] for i in x) if x!={} else "")
for g in top_cast_characters:
    test['cast_character_'+g] = test['all_cast_character'].apply(lambda x: 1 if g in x else 0)

**Crew**

In [341]:
train['crew'].apply(lambda x: len(x) if x!={} else 0).value_counts().head(10), train['crew'].apply(lambda x: len(x) if x!={} else 0).value_counts().tail(10)

Usually there is 2 to 15 crew but there are also exist some data that have more then 100 crew

In [342]:
list_of_crew_names = train['crew'].apply(lambda x: [i['name'] for i in x] if x!={} else [])
Counter([i for j in list_of_crew_names for i in j]).most_common(15)

In [343]:
train['num_crew'] = train['crew'].apply(lambda x: len(x) if x!={} else 0)
train['all_crew_name'] = train['crew'].apply(lambda x: ' '.join(i['name'] for i in x) if x!={} else [])
top_crew_names = [m[0] for m in Counter([i for j in list_of_crew_names for i in j]).most_common(15)]

for g in top_crew_names:
    train['crew_name_' + g] = train['all_crew_name'].apply(lambda x: 1 if g in x else 0)


test['num_crew'] = test['crew'].apply(lambda x: len(x) if x!={} else 0)
test['all_crew_name'] = test['crew'].apply(lambda x: ' '.join(i['name'] for i in x) if x!={} else [])

for g in top_crew_names:
    test['crew_name_' + g] = test['all_crew_name'].apply(lambda x: 1 if g in x else 0)

**Genders for crew**

In [344]:
list_of_gender = train['crew'].apply(lambda x: [i['gender'] for i in x] if x!={} else [])
Counter(i for j in list_of_gender for i in j).most_common(15)

In [345]:
train['crew_gender_0'] = train['cast'].apply(lambda x: sum(1 for i in x if i['gender'] == 0))
train['crew_gender_1'] = train['cast'].apply(lambda x: sum(1 for i in x if i['gender'] == 0))
train['crew_gender_2'] = train['cast'].apply(lambda x: sum(1 for i in x if i['gender'] == 2))

test['crew_gender_0'] = test['cast'].apply(lambda x: sum(1 for i in x if i['gender'] == 0))
test['crew_gender_1'] = test['cast'].apply(lambda x: sum(1 for i in x if i['gender'] == 0))
test['crew_gender_2'] = test['cast'].apply(lambda x: sum(1 for i in x if i['gender'] == 2))

Here we just assigned number of gender in total crew for particular one movie like number of male for x movie from all x cast

Job of crew and number of jobs

In [346]:
list_of_jobs = train['crew'].apply(lambda x: [i['job'] for i in x] if x!={} else [])
Counter([i for j in list_of_jobs for i in j]).most_common(20)

In [347]:
train['all_job'] = train['crew'].apply(lambda x: [i['job'] for i in x] if x!={} else [])
top_job = [m[0] for m in Counter([i for j in list_of_jobs for i in j]).most_common(20)]

for g in top_job:
    train['job_'+g] = train['all_job'].apply(lambda x: 1 if g in x else 0)

test['all_job'] = test['crew'].apply(lambda x: [i['job'] for i in x] if x!={} else [])

for g in top_job:
    test['job_'+g] = test['all_job'].apply(lambda x: 1 if g in x else 0)

**Data Visualization**

In [348]:
fig, ax = plt.subplots(1,2, figsize=(16, 6))

plt.subplot(1, 2, 1)
plt.hist(train['revenue'])
plt.title("Revenue distribution")

plt.subplot(1, 2, 2)
plt.hist(np.log1p(train['revenue']))
plt.title("Log1 revenue distribution")

plt.show()

When we apply log on revenue then it becomes less skewed and it will be better for our model

In [349]:
train['log_revenue'] = np.log1p(train['revenue'])

In [350]:
train['budget'].skew(), np.log1p(train['budget']).skew()

In [351]:
fig, ax = plt.subplots(1,2, figsize=(16, 6))

plt.subplot(1,2,1)
plt.hist(train['budget'])
plt.title("Budget distribution")

plt.subplot(1,2,2)
plt.hist(np.log1p(train['budget']))
plt.title("Log1p Budget distribution")

plt.show()

In [352]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

plt.subplot(1, 2, 1)
sns.scatterplot(x = np.log(train['budget']), y=train['log_revenue'])
plt.xlabel('log_budget')
plt.title("log Revenue vs log budget")

plt.subplot(1,2,2)
sns.scatterplot(x='budget', y='log_revenue', data=train)
plt.title("log revenue vs budget")

plt.show()

Scatter plot of log_revenue vs log_budget is very good as compared to log_revenue vs budget

In [353]:
train['log_budget'] = np.log1p(train['budget'])
test['log_budget'] = np.log1p(test['budget'])

In [354]:
train['homepage'].apply(lambda x: 1 if x!=np.nan else 0).value_counts()

In [355]:
plt.subplots(1,2,figsize=(16, 6))
plt.subplot(1,2,1)
sns.boxenplot(x='original_language', y='revenue', data= train.loc[train['original_language'].isin(train['original_language'].value_counts().head(10).index)] )
plt.title("Revenue vs Original language")

plt.subplot(1,2,2)
sns.boxenplot(x='original_language', y='log_revenue', data= train.loc[train['original_language'].isin(train['original_language'].value_counts().head(10).index)] )
plt.title("Log Revenue vs Original language")

plt.show()

Based on previous three graph we can completely say that log_revenue is really good choice for our model as compared to revenue

In [356]:
plt.figure(figsize=(9, 9))
title = ' '.join(train['original_title'].values)
img = WordCloud(width=1000, height=1000, background_color='white').generate(title)
plt.title("Original title wordcloud")
plt.imshow(img)
plt.axis('off')
plt.show()

Most occuring words in movie titles are Man and Last.

In [357]:
plt.figure(figsize=(9,9))
Overview = ' '.join(train['overview'].fillna(' ').values)
img = WordCloud(width=1000, height=1000, background_color='white').generate(Overview)
plt.imshow(img)
plt.title("Overview wordcloud")
plt.show()

Let's see impact of words on revenue

In [358]:
tfidf = TfidfVectorizer()

x = tfidf.fit_transform(train['overview'].fillna(''))
y = train['log_revenue']

X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.1)

lr = LinearRegression()
lr.fit(X_train, y_train)

In [359]:
mean_squared_error(lr.predict(X_valid), y_valid), mean_absolute_error(lr.predict(X_valid), y_valid)

In [360]:
eli5.show_weights(lr, vec=tfidf, top=20, feature_filter=lambda x: x != '<BIAS>')

In [361]:
print("Target value : ", train['log_revenue'][500])
eli5.show_prediction(lr, doc=train['overview'].values[500], vec = tfidf)

**Popularity**

In [362]:
plt.subplots(1,2,figsize=(16, 6))

plt.subplot(1,2,1)
plt.scatter(x='popularity', y='revenue', data=train)
plt.title("Revenue vs Popularity")

plt.subplot(1,2,2)
plt.scatter(x='popularity', y='log_revenue', data=train)
plt.title("Log revenue vs popularity")

plt.show()

Based on scatter plot we can say revenue is not much dependent on popularity

**Release Date**

In [363]:
test[test.release_date.isnull()].index

In [364]:
test.at[828, 'release_date'] = '1/5/2000'

In [365]:
train['release_date_year'] = train['release_date'].apply(lambda x: 19*100 + int(x.split('/')[-1]) if int(x.split('/')[-1]) > 22 else 20*100 + int(x.split('/')[-1]))
test['release_date_year'] = test['release_date'].apply(lambda x: 19*100 + int(x.split('/')[-1]) if int(x.split('/')[-1]) > 22 else 20*100 + int(x.split('/')[-1]))

train['release_date_month'] = pd.to_datetime(train['release_date']).dt.month
test['release_date_month'] = pd.to_datetime(test['release_date']).dt.month

train['release_date_weekday'] = pd.to_datetime(train['release_date']).dt.weekday
test['release_date_weekday'] = pd.to_datetime(test['release_date']).dt.weekday

train['release_date_weekofyear'] = pd.to_datetime(train['release_date']).dt.weekofyear
test['release_date_weekofyear'] = pd.to_datetime(test['release_date']).dt.weekofyear

train['release_date_day'] = pd.to_datetime(train['release_date']).dt.day
test['release_date_day'] = pd.to_datetime(test['release_date']).dt.day

In [366]:
d1 = train['release_date_year'].value_counts().sort_index()
d2 = test['release_date_year'].value_counts().sort_index()

data = [go.Scatter(x = d1.index, y=d1.values, name='train'), go.Scatter(x = d2.index, y = d2.values, name='test')]
layout = go.Layout(dict(title="Number of movies per year",
                  xaxis = dict(title="year"),
                  yaxis = dict(title="Count")),legend = dict(orientation='v'))
py.iplot(dict(data = data, layout=layout))

# d1 = train['release_date_year'].value_counts().sort_index()
# d2 = test['release_date_year'].value_counts().sort_index()
# data = [go.Scatter(x=d1.index, y=d1.values, name='train'), go.Scatter(x=d2.index, y=d2.values, name='test')]
# layout = go.Layout(dict(title = "Number of films per year",
#                   xaxis = dict(title = 'Year'),
#                   yaxis = dict(title = 'Count'),
#                   ),legend=dict(
#                 orientation="v"))
# py.iplot(dict(data=data, layout=layout))

In [367]:
d1 = train['release_date_year'].value_counts().sort_index()
d2 = test['release_date_year'].value_counts().sort_index()
data = [go.Scatter(x=d1.index, y=d1.values, name='train'), go.Scatter(x=d2.index, y=d2.values, name='test')]
layout = go.Layout(dict(title = "Number of films per year",
                  xaxis = dict(title = 'Year'),
                  yaxis = dict(title = 'Count'),
                  ),legend=dict(
                orientation="v"))
py.iplot(dict(data=data, layout=layout))

In [368]:
d1 = train['release_date_year'].value_counts().sort_index()
d2 = train.groupby('release_date_year')['revenue'].sum()

data = [go.Scatter(x=d1.index, y=d1.values, name='film count'), go.Scatter(x=d2.index, y=d2.values, name='total revenue', yaxis='y2')]
layout = go.Layout(dict(title = "Number of films and total revenue per year",
                  xaxis = dict(title = 'Year'),
                  yaxis = dict(title = 'Count'),
                  yaxis2=dict(title='Total revenue', overlaying='y', side='right')
                  ),legend=dict(
                orientation="v"))
py.iplot(dict(data=data, layout=layout))

In [369]:
d1 = train['release_date_year'].value_counts().sort_index()
d2 = train.groupby(['release_date_year'])['revenue'].mean()
data = [go.Scatter(x=d1.index, y=d1.values, name='film count'), go.Scatter(x=d2.index, y=d2.values, name='mean revenue', yaxis='y2')]
layout = go.Layout(dict(title = "Number of films and average revenue per year",
                  xaxis = dict(title = 'Year'),
                  yaxis = dict(title = 'Count'),
                  yaxis2=dict(title='Average revenue', overlaying='y', side='right')
                  ),legend=dict(
                orientation="v"))
py.iplot(dict(data=data, layout=layout))

In [370]:
sns.catplot(x='release_date_weekday', y='revenue', data=train);
plt.title('Revenue on different days of week of release');

Films releases on Wednesdays and on Thursdays tend to have a higher revenue.

In [371]:
fig, ax = plt.subplots(1, 3, figsize=(20, 6))

plt.subplot(1,3,1)
plt.hist(train['runtime'].fillna(0)/60, bins=20)
plt.title("Distribution of runtime")

plt.subplot(1,3,2)
plt.scatter(x = 'runtime', y='revenue', data=train)
plt.title("Revenue vs runtime")

plt.subplot(1,3,3)
plt.scatter(x = 'runtime', y='popularity', data=train)
plt.title("Revenue vs popularity")

plt.show()

In [372]:
train['status'].value_counts()

In [373]:
test['status'].value_counts()

**Tagline**

In [374]:
text = ' '.join(train['tagline'].fillna("").values)
img = WordCloud(width=1000, height=1000, background_color='white').generate(text)
plt.figure(figsize=(9, 9))
plt.imshow(img)
plt.title("Tagline wordcloud")
plt.axis("off")
plt.show()

In [375]:
f, axes = plt.subplots(3, 5, figsize=(24, 12))
plt.suptitle('Violinplot of revenue vs genres')
for i, e in enumerate([col for col in train.columns if 'genres_' in col]):
    sns.violinplot(x=e, y='revenue', data=train, ax=axes[i // 5][i % 5]);

Family, adventure, fantasy and animation have high revenue

In [376]:
fig, ax = plt.subplots(4, 5, figsize=(21, 18))
fig.tight_layout()
plt.subplots_adjust(wspace=None, hspace=None)
plt.axis('off')
plt.suptitle("Violin plot of revenue vs production companies")
for i,e in enumerate([col for col in train.columns if 'production_companies_' in col]):
    sns.violinplot(x = e, y='revenue', data = train, ax = ax[i // 5][i % 5])

In [377]:
f, axes = plt.subplots(4, 5, figsize=(24, 24))
plt.suptitle('Violinplot of revenue vs production country')
for i, e in enumerate([col for col in train.columns if 'production_countries_' in col]):
    sns.violinplot(x=e, y='revenue', data=train, ax=axes[i // 5][i % 5]);

In [378]:
plt.figure(figsize=(16, 8))

plt.subplot(1, 2, 1)
plt.scatter(train['num_cast'], train['revenue'])
plt.title('Number of cast members vs revenue');
plt.xlabel("Number of cast")
plt.ylabel("Revenue")

plt.subplot(1, 2, 2)
plt.scatter(train['num_cast'], train['log_revenue'])
plt.title('Number of cast members vs log revenue');
plt.xlabel("Number of cast")
plt.ylabel("Log Revenue")

plt.show()

number of cast and log_revenue is positively correlated with eachother

In [379]:
fig, ax = plt.subplots(nrows = 4, ncols = 5, figsize=(24, 18))

for i,e in enumerate([col for col in train.columns if 'cast_name_' in col]):
    sns.violinplot(x = e, y = 'revenue', data= train, ax = ax[i//5][i%5])

In [380]:
len([col for col in train.columns if 'cast_character_' in col])

In [381]:
fig, ax = plt.subplots(4, 5, figsize=(24, 18))
for i,e in enumerate([col for col in train.columns if 'cast_character_' in col]):
    sns.violinplot(x = e, y = 'revenue', data=train, ax = ax[i//5][i%5])

In [382]:
plt.subplots(1, 2, figsize=(18, 6))

plt.subplot(1,2,1)
sns.scatterplot(x = 'num_crew', y = 'revenue', data=train)
plt.title("Revenue vs number of crew")
plt.xlabel("Number of crew")
plt.ylabel("Revenue")

plt.subplot(1,2, 2)
sns.scatterplot(x = 'num_crew', y = 'log_revenue', data=train)
plt.title("log_revenue vs number of crew")
plt.xlabel("Number of crew")
plt.ylabel("log_revenue")

plt.show()

In [383]:
train.head(1)

Here i deleted column that contains unique values and columns that i already took data from them

In [384]:
train.drop(['genres', 'homepage', 'imdb_id', 'poster_path', 'production_companies','production_countries','release_date', 'spoken_languages','status', 'Keywords','cast','crew', 'id'], axis=1, inplace=True)
test.drop(['genres', 'homepage', 'imdb_id', 'poster_path', 'production_companies','production_countries','release_date', 'spoken_languages','status', 'Keywords','cast','crew', 'id'], axis=1, inplace=True)

In [385]:
train['collection'] = train['collection'].apply(lambda x: str(x))
test['collection'] = test['collection'].apply(lambda x: str(x))

Using label encoder assign unique number to all unique values

In [386]:
for col in ['original_language', 'all_genres', 'collection']:
    lb = LabelEncoder()
    lb.fit(list(train[col].fillna("")) + list(test[col].fillna("")))
    train[col] = lb.transform(train[col].fillna(""))
    test[col] = lb.transform(test[col].fillna(""))

In [387]:
for col in ['title', 'tagline', 'overview', 'original_title']:
    train['len_' + col] = train[col].fillna('').apply(lambda x: len(str(x)))
    train['words_' + col] = train[col].fillna('').apply(lambda x: len(str(x.split(' '))))
    train = train.drop(col, axis=1)
    test['len_' + col] = test[col].fillna('').apply(lambda x: len(str(x)))
    test['words_' + col] = test[col].fillna('').apply(lambda x: len(str(x.split(' '))))
    test = test.drop(col, axis=1)

Again dropped unnecessary columns

In [388]:
train.drop(['all_production_companies', 'all_production_countries', 'all_spoken_languages', 'all_Keywords', 'all_cast', 'all_cast_character', 'all_crew_name', 'all_job'], axis=1, inplace=True)
test.drop(['all_production_companies', 'all_production_countries', 'all_spoken_languages', 'all_Keywords', 'all_cast', 'all_cast_character', 'all_crew_name', 'all_job'], axis=1, inplace=True)

In [389]:
train.head(2)

In [390]:
test.head(2)

In [391]:
train.shape, test.shape

In [392]:
print("Columns that have null values")
for col in test.columns:
    if train[col].isnull().sum() > 0:
        print(col)

In [393]:
print("Columns that are not in test but in train")
for col in train.columns:
    if col not in test.columns:
        print(col)

In [394]:
train.drop(['budget'], axis=1, inplace=True)
test.drop(['budget'], axis=1, inplace=True)

In [395]:
train['runtime'].fillna(train['runtime'].mean(), inplace=True)

Drop columns that have only one unique value

In [396]:
for col in train.columns:
    if(train[col].var() == 0):
        print(col)
        train.drop(col, axis=1, inplace=True)
        test.drop(col, axis=1, inplace=True)

Y as log_revenue our dependent variable

X our input features

In [397]:
X = train.drop(['log_revenue', 'revenue'], axis=1)
Y = train['log_revenue']

Divide dataset into train and valid for validation purpose

In [398]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.1, random_state = 42)
X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape

**Linear Regression**

In [399]:
lr = LinearRegression()
lr.fit(X_train, Y_train)
mean_squared_error(lr.predict(X_valid), Y_valid), mean_absolute_error(lr.predict(X_valid), Y_valid)

**Random Forest**

In [400]:
rf = RandomForestRegressor()
rf.fit(X_train, Y_train)
mean_squared_error(rf.predict(X_valid), Y_valid), mean_absolute_error(rf.predict(X_valid), Y_valid)

Feature Importance

In [401]:
ax = plt.figure(figsize=(12, 10))
feat_importances = pd.Series(rf.feature_importances_, index=X_train.columns)
feat_importances.nlargest(30).plot(kind='barh')

output feature is highly dependent on log_budget, populrity and release_date_year

**Ada boost**

In [402]:
ada = AdaBoostRegressor()
ada.fit(X_train, Y_train)
mean_squared_error(ada.predict(X_valid), Y_valid), mean_absolute_error(ada.predict(X_valid), Y_valid)

**Gradient Boosting regression**

In [403]:
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

reg = GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)
mean_squared_error(reg.predict(X_valid), Y_valid), mean_absolute_error(reg.predict(X_valid), Y_valid)

**XGB Regressor**

In [404]:
xgb = XGBRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(xgb, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

**LGBM Regressor**

In [405]:
params = {'num_leaves': 30,
         'min_data_in_leaf': 20,
         'objective': 'regression',
         'max_depth': 5,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.2,
         "verbosity": -1}

lgb = LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)
lgb.fit(X_train, Y_train)
mean_squared_error(lgb.predict(X_valid), Y_valid), mean_absolute_error(lgb.predict(X_valid), Y_valid)

In [406]:
eli5.show_weights(lgb, feature_filter=lambda x: x != '<BIAS>')

RandomForest and LGBMRegressor perform best as per there mean absolute error